# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 2.1.1
- **Date :** 09/11/2023
- **Heure :** 14:36
- **Données :** `ready_Evaporation_Sunshine.csv`
- **Tri :** par défaut
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** KNN + DTW

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../../data/processed/omar/ready_Evaporation_Sunshine.csv", index_col = 1)
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,2008,12,1,0,3.141593,3.141593,2.748894,-36.080477,146.91628,0
2008-12-02,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,2008,12,2,0,2.748894,1.963495,3.534292,-36.080477,146.91628,0
2008-12-03,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,2008,12,3,0,3.534292,3.141593,3.534292,-36.080477,146.91628,0
2008-12-04,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,2008,12,4,0,0.785398,5.497787,0.000000,-36.080477,146.91628,0
2008-12-05,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,2008,12,5,0,3.141593,0.392699,2.356194,-36.080477,146.91628,0


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10453 entries, 2008-12-01 to 2017-06-24
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         10453 non-null  float64
 1   MaxTemp         10453 non-null  float64
 2   Rainfall        10453 non-null  float64
 3   WindGustSpeed   10453 non-null  float64
 4   WindSpeed9am    10453 non-null  float64
 5   WindSpeed3pm    10453 non-null  float64
 6   Humidity9am     10453 non-null  float64
 7   Humidity3pm     10453 non-null  float64
 8   Pressure9am     10453 non-null  float64
 9   Pressure3pm     10453 non-null  float64
 10  Cloud9am        10453 non-null  float64
 11  Cloud3pm        10453 non-null  float64
 12  Temp9am         10453 non-null  float64
 13  Temp3pm         10453 non-null  float64
 14  RainToday       10453 non-null  int64  
 15  RainTomorrow    10453 non-null  int64  
 16  year            10453 non-null  int64  
 17  month           10453 

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Importation de la classe `KNeighborsTimeSeriesClassifier`

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [10]:
# Instanciation d'un modèle classificateur avec `n_neighbors = 1` et `distance = "dtw"`

clf_knn_ts = KNeighborsTimeSeriesClassifier(n_neighbors = 1, distance = "dtw")

In [11]:
# Récupération des paramètres initiaux

clf_knn_ts.get_params()

{'algorithm': 'brute',
 'distance': 'dtw',
 'distance_mtype': None,
 'distance_params': None,
 'leaf_size': 30,
 'n_jobs': None,
 'n_neighbors': 1,
 'pass_train_distances': False,
 'weights': 'uniform'}

In [12]:
# Entraînement du modèle

clf_knn_ts.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier()

In [13]:
# Récupération des paramètres ajustés

clf_knn_ts.get_fitted_params()

{'classes': array([0, 1]),
 'fit_time': 226,
 'knn_estimator': KNeighborsClassifier(algorithm='brute', metric='precomputed', n_neighbors=1),
 'n_classes': 2,
 'knn_estimator__classes': array([0, 1]),
 'knn_estimator__effective_metric': 'precomputed',
 'knn_estimator__effective_metric_params': {},
 'knn_estimator__n_features_in': 8711,
 'knn_estimator__n_samples_fit': 8711,
 'knn_estimator__outputs_2d': False}

In [14]:
# Réalisation des prédictions

y_pred = clf_knn_ts.predict(X_test)

## 5. Évaluation

In [15]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,1448,135
1,98,61


In [16]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      1583
           1       0.31      0.38      0.34       159

    accuracy                           0.87      1742
   macro avg       0.62      0.65      0.63      1742
weighted avg       0.88      0.87      0.87      1742

